In [ ]:
# Cell 14: feature engineering recap and save subset
features = ["timestamp","coin","price","return_1h","log_return_1h","ma_24h","volatility_24h","momentum_24h","total_volume","market_cap"]
feat_df = df[features].copy()
feat_df.to_csv("../data/features_export.csv", index=False)
feat_df.head()


In [ ]:
# Cell 13: simple anomaly detection
btc = df[df["coin"] == "bitcoin"].set_index("timestamp").sort_index()
# z-score approach on log_return_1h
btc["z_return"] = (btc["log_return_1h"] - btc["log_return_1h"].mean()) / btc["log_return_1h"].std()
btc["anomaly_z"] = btc["z_return"].abs() > 3  # typical 3-sigma threshold

# Isolation Forest
iso = IsolationForest(contamination=0.001, random_state=42)
valid = btc["log_return_1h"].dropna().values.reshape(-1,1)
iso.fit(valid)
btc.loc[btc["log_return_1h"].dropna().index, "iso_score"] = iso.decision_function(valid)
btc.loc[btc["log_return_1h"].dropna().index, "iso_anomaly"] = iso.predict(valid) == -1

# Show flagged rows
anomalies = btc[(btc["anomaly_z"]) | (btc["iso_anomaly"]==True)]
anomalies.head(20)


In [ ]:
# Cell 12: quick ARIMA on log returns (toy demo)
from statsmodels.tsa.arima.model import ARIMA

series = btc["log_return_1h"].dropna().asfreq("H")
# Use a tiny model (p=1,d=0,q=1) for demonstration — returns often don't need differencing
model = ARIMA(series.iloc[-24*10:], order=(1,0,1))  # last 10 days to speed up
res = model.fit()
print(res.summary())
pred = res.get_forecast(steps=24)
pred_mean = pred.predicted_mean
pred_ci = pred.conf_int()


In [ ]:
# Cell 11: OLS regression - does volume predict volatility?
daily = df.set_index("timestamp").groupby("coin").resample("1D").agg({
    "log_return_1h": "std",
    "total_volume": "sum"
}).reset_index().rename(columns={"log_return_1h":"daily_volatility", "total_volume":"daily_volume"})

btc_daily = daily[daily["coin"] == "bitcoin"].dropna()
X = np.log1p(btc_daily["daily_volume"])  # log transform volume
X = sm.add_constant(X)
y = btc_daily["daily_volatility"]
model = sm.OLS(y, X).fit()
print(model.summary())


In [ ]:
# Cell 10: hypothesis test - are weekend returns larger (abs) than weekday?
df["is_weekend"] = df["timestamp"].dt.dayofweek >= 5
btc = df[df["coin"] == "bitcoin"]
weekend = btc[btc["is_weekend"]]["return_1h"].dropna().abs()
weekday = btc[~btc["is_weekend"]]["return_1h"].dropna().abs()

from scipy.stats import mannwhitneyu
stat, p = mannwhitneyu(weekend, weekday, alternative="two-sided")
print("Mann-Whitney U test statistic:", stat, "p-value:", p)


In [ ]:
# Cell 9: correlation heatmap across log returns for top coins
pivot = df.pivot_table(index="timestamp", columns="coin", values="log_return_1h")
corr = pivot.corr()
plt.figure(figsize=(8,6))
sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Correlation matrix (log returns)")
plt.show()


In [ ]:
# Cell 8: volatility clustering - plot absolute returns
plt.figure(figsize=(14,4))
plt.plot(btc.index, btc["log_return_1h"].abs(), label="|log return|")
plt.title("Absolute log returns (proxy for volatility) - shows clustering")
plt.show()


In [ ]:
# Cell 7: ADF test on price and log returns
def adf_test(series):
    res = adfuller(series.dropna(), autolag='AIC')
    return {"adf_stat": res[0], "pvalue": res[1], "usedlag": res[2], "nobs": res[3]}

print("ADF on price:", adf_test(btc["price"]))
print("ADF on log returns:", adf_test(btc["log_return_1h"]))


In [ ]:
# Cell 6: ACF & PACF on log returns (stationarity check)
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
series = btc["log_return_1h"].dropna()

plt.figure(figsize=(12,4))
plot_acf(series, lags=48, alpha=0.05)
plt.title("ACF of Bitcoin 1h log returns (48 lags)")
plt.show()

plt.figure(figsize=(12,4))
plot_pacf(series, lags=48, alpha=0.05, method='ywm')
plt.title("PACF of Bitcoin 1h log returns (48 lags)")
plt.show()


In [ ]:
# Cell 5: price and moving averages
coin = "bitcoin"
btc = df[df["coin"] == coin].set_index("timestamp").sort_index()

plt.figure(figsize=(14,6))
plt.plot(btc.index, btc["price"], label="price")
plt.plot(btc.index, btc["ma_24h"], label="24H MA", linewidth=2)
plt.title(f"{coin} price and 24h moving average")
plt.legend()
plt.show()


In [ ]:
# Cell 4: return distributions
plt.figure(figsize=(12,6))
sns.histplot(df, x="return_1h", hue="coin", element="step", stat="density", common_norm=False, bins=200)
plt.xlim(-0.5, 0.5)
plt.title("Distribution of 1-hour returns by coin (zoomed)")
plt.xlabel("1-hour return")
plt.ylabel("Density")
plt.show()


In [ ]:
# Cell 3: per-coin stats
summary = df.groupby("coin").agg(
    observations=("price", "count"),
    avg_price=("price", "mean"),
    median_price=("price", "median"),
    std_price=("price", "std"),
    avg_volume=("total_volume", "mean")
).reset_index()
summary


In [ ]:
# Cell 2: sanity checks
print("n rows:", len(df))
print("coins:", df['coin'].unique())
print("time range:", df['timestamp'].min(), "to", df['timestamp'].max())

# check missingness
missing_summary = df.isna().mean().round(4) * 100  # percent missing per column
print("percent missing per column:\n", missing_summary)

# Remove rows with missing price (if any remain)
df = df.dropna(subset=["price"]).copy()


In [ ]:
# Cell 1: imports and load
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from sklearn.ensemble import IsolationForest
from sqlalchemy import create_engine, text

# Create engine and load data
engine = create_engine("sqlite:///../data/crypto_data.sqlite", future=True)
df = pd.read_sql(text("SELECT * FROM crypto_prices"), engine, parse_dates=["timestamp", "ingested_at"])
df.head()
